In [5]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['LM(한국어)', '문서 요약']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            doc_id = line['dataset']['doc_id']
            doc_type = line['dataset']['doc_type']
            doc_title_kr = line['dataset']['doc_title']['kor']
            doc_title_en = line['dataset']['doc_title']['eng']
            
            #### data preprocess end 
            summaries = []
            for context in line['dataset']['context_info']:
                context_id = context['context_id']
                text = context['context']
                summary = context['summary']
                summaries.append({'context_id': context_id, 'text': text, 'summary': summary})

            #### LM(한국어)
            data = {'text': None}
            ## preprocess data from line
            text = '##type : ' + doc_type + '\n'
            if doc_title_kr:
                text += '##title_kr : ' + doc_title_kr + '\n'
            if doc_title_en:
                text += '##title_en : ' + doc_title_en + '\n'
            text += '##text : ' + '\n\n'.join([summary['text'] for summary in summaries])
            text += '\n\n##summary : ' + '\n\n'.join([summary['summary'] for summary in summaries])
            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### LM(한국어) end
            

            #### 문서 요약
            for summary in summaries:
                data = {'input': {}, 'output': None}
                ## preprocess data from line
                data['input']['doc_id'] = doc_id
                data['input']['doc_type'] = doc_type
                data['input']['doc_title_kr'] = doc_title_kr
                data['input']['doc_title_en'] = doc_title_en
                data['input']['context_id'] = summary['context_id']
                data['input']['text'] = summary['text']
                data['output'] = summary['summary']
                ## preprocess data from line end
                task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### 문서 요약 end
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 4/4 [00:01<00:00,  3.93it/s]
